# Modules

In [63]:
import pandas as pd # routine analysis with dataframes
import numpy as np # routine array analysis
import pickle # reading in pickle files (PCA morphospace file)
from scipy.spatial import ConvexHull # for calculating convex hull of morphospace
from scipy.spatial import Delaunay # for delauney triangulation functions
from scipy.stats import dirichlet # for sampling simplices in the morphospace
from numpy.linalg import det # for sampling simplices in the morphospace
import math # for math functions
import os # for directory functions

# Functions

In [64]:
def save_params(keys, values, path):

    """
    Given lists of parameter variable names and values, saves as csv file to desired path
    
    Inputs 
    keys: a list of keys, parameter names
    values: a list of values, parameter values
    path: a filename path to save the param values as .csv file

    Outputs:
    No return, saves the csv to the desired path
    """
    
    # convert values into lists
    list_vals = [[x] for x in values]
    
    # convert parameter names and values into dictionary
    data = dict(zip(keys, list_vals))
    
    # convert dict into df
    df = pd.DataFrame.from_dict(data)
    
    # save param values
    df.to_csv(path, index=False)

def set_params(path):

    """
    Given a path, read in a csv file with parameter variable names and values
    
    Inputs 
    path: a filename path to read in a csv 

    Outputs:
    Sets global variables and values from the csv file
    Returns a dictionary of parameter names and values
    """
    
    # read in param values from csv
    df = pd.read_csv(path)
    
    # create variables for parameters
    for column in df.columns:
        globals()[column] = df[column].item()

    # create a dictionary of parameter variables to return
    param_var_list = [
        "parameter_name", 
        "morphospace_file", 
        "PC_val_file",
        "PCa", 
        "PCb",
        "grid_density",
        "low_lf_num",
        "high_lf_num",
        "low_juv_len",
        "high_juv_len",
        "low_adult_len",
        "high_adult_len",
        "lf_curve",
        "lf_curl",
        "max_down_curl",
        "max_up_curl",
        "ang_min_limit",
        "ang_max_limit",
        "phyllo_option"]

    param_val_list = [
        parameter_name, 
        morphospace_file, 
        PC_val_file,
        PCa, 
        PCb,
        grid_density,
        low_lf_num,
        high_lf_num,
        low_juv_len,
        high_juv_len,
        low_adult_len,
        high_adult_len,
        lf_curve,
        lf_curl,
        max_down_curl,
        max_up_curl,
        ang_min_limit,
        ang_max_limit,
        phyllo_option]
    
    # convert parameter names and values into dictionary
    param_var_dict = dict(zip(param_var_list, param_val_list))

    return param_var_dict

def dist_in_hull(points, n):

    """
    From: https://stackoverflow.com/questions/59073952/how-to-get-uniformly-distributed-points-in-convex-hull
    Accessed 11 February 2024
    
    Given a set of points, find delauney simplices of hull and randomly sample n points within

    Inputs:
    points: an array of point values, any dimensions
    n: the number of points to sample within the convex hull

    Outputs
    coordinates of randomly sampled points (array)
    """
    
    dims = points.shape[-1]
    hull = points[ConvexHull(points).vertices]
    deln = hull[Delaunay(hull).simplices]

    vols = np.abs(det(deln[:, :dims, :] - deln[:, dims:, :])) / math.factorial(dims)    
    sample = np.random.choice(len(vols), size = n, p = vols / vols.sum())

    return np.einsum('ijk, ij -> ik', deln[sample], dirichlet.rvs([1]*(dims + 1), size = n))

def select_leaf_number(low_lf_num, high_lf_num):
    """
    Given low and high leaf number values, uniformly sample a leaf number for the plant
    
    Inputs
    low_lf_num: integer value of lowest possible leaf number (inclusive)
    high_lf_num: integer value of highest possible leaf number (not included)

    Ouputs
    n: the selected leaf number uniformly sampled from provided range
    """

    return np.random.randint(low_lf_num, high_lf_num) 

def select_phyllotaxy_angle(phyllo_option):
    """
    Given a phyllotaxy angle option value (see below) set phyllotaxy angle

    Inputs
    phyllo_option: integer value selecting phyllotaxy option

    Outputs
    phyllo_ang: float value of calculated phyllotaxy angle
    """

    # 0 = [-137.5077640500378546463487,137.5077640500378546463487]
    # 1 = [-137.5077640500378546463487,137.5077640500378546463487,90,180]
    # 2 = [-137.5077640500378546463487,137.5077640500378546463487,90,180,<random -360 to +360>]
    # 3 = [<random -360 to +360>]

    if phyllo_option==0:
        phyllo_ang=np.random.choice(np.array([-137.5077640500378546463487,137.5077640500378546463487]))
    elif phyllo_option==1:
        phyllo_ang=np.random.choice(np.array([-137.5077640500378546463487,137.5077640500378546463487,90,180]))
    elif phyllo_option==2:
        phyllo_ang=np.random.choice(np.array([-137.5077640500378546463487,137.5077640500378546463487,90,180,np.random.uniform(-360,360)]))
    else:
        phyllo_ang=np.random.uniform(-360,360)

    return phyllo_ang

def select_leaf_lengths(low_juv_len, high_juv_len, low_adult_len, high_adult_len):
    """
    Given relative leaf lengths of start (juvenile) and end (adult) of leaf series, set relative leaf lengths

    Inputs
    low_juv_len: low limit juvenile leaf length (relative to 1)
    high_juv_len: high limit juvenile leaf length (relative to 1)
    low_adult_len: low limit adult leaf length (relative to 1)
    high_adult_len: high limit adult leaf length (relative to 1)

    Outputs
    juv_len: the relative length to 1 of juvenile leaf
    adult_len: the relative length to 1 of the adult leaf
    node_max: the node in the leaf series of the max leaf height (from 0 to 1)
    """
    # select relative height of first leaf in series
    juv_len = np.random.uniform(low_juv_len, high_juv_len) 
    
    # set node position of max leaf height of 1 (0 to 1)
    node_max = np.random.uniform(0.1, 0.9) # a little bit in from 0 and 1 at 0.1 to 0.9
    
    # set the relative height of last leaf in series
    adult_len = np.random.uniform(low_adult_len, high_adult_len) 

    return juv_len, adult_len, node_max

def select_leaf_surface_variables(lf_curve, lf_curl, max_down_curl, max_up_curl):
    """
    Given curvature and curling scaling values and positioning of curling, return leaf surface (z axis) values

    Inputs:
    lf_curve: scaling factor for the amount of leaf curvature (relative to leaf margins)
    lf_curl: scaling factor for the amount of leaf curling (from base to tip)
    max_down_curl: a 0 to 1 value of maximum downward curl
    max_up_curl: a 0 to 1 value of maximum upward curl

    Outputs:
    leaf_curve: the leaf curvature scaling amount
    leaf_curl: the leaf curling scaling amount
    leaf_tip: the leaf curling value at the tip of the leaf
    leaf_mid: the leaf curling value at the mid point of the leaf length
    """
    # calculate the scaling factor for leaf curvature and leaf curl
    leaf_curve = np.random.uniform(0,lf_curve) 
    leaf_curl = np.random.uniform(0,lf_curl) 
    
    # calculate leaf curl amount (z value)
    leaf_tip = np.random.uniform(max_down_curl,max_up_curl) 
    
    # set middle of leaf curl (also z value, but curl at mid point of length)
    leaf_mid = np.random.uniform(0,leaf_tip) 

    return leaf_curve, leaf_curl, leaf_tip, leaf_mid

def select_leaf_elevation_angles(ang_min_limit, ang_max_limit):
    """
    Given the minimum and maximum leaf angle limits, calculate the leaf elevation limits

    Inputs
    ang_min_limit: the minimum angle limit (in the context of 0 to the limit)
    ang_max_limit: the maximum angle limit (in the context of the selected min angle to the max limit)

    Outputs
    angle_min: the selected minimum angle value
    angle_max: the selected maximum angle value
    """
    # calculate elevation angles
    angle_min = np.random.uniform(0,ang_min_limit) # get the minimum angle
    angle_max = np.random.uniform(angle_min,ang_max_limit) # get the maximum angle\

    return angle_min, angle_max

def select_morphospace_points(morphospace_file, PC_val_file, PCa, PCb):
    """
    Given a morphospace and PC values select morphospace points to generate a leaf shape series

    Inputs
    morphospace_file: path to a morphospace file (.pkl)
    PC_val_file: path to PC value file (.npy)
    PCa: the first PC axis to consider (index starts at 0)
    PCb: the second PC axis to consider (index starts at 0)

    Outputs:
    start_x: the x value (PCa) of the leaf series start
    start_y: the y value (PCb) of the leaf series start
    end_x: the x value (PCa) of the leaf series end
    end_y: the y value (PCb) of the leaf series end
    """
    # find the two points in PCA space to make the leaf series
    # Load PCA model
    with open(morphospace_file, 'rb') as file:
        pca = pickle.load(file)
    # Load in PC values
    PCs = np.load(PC_val_file)
    
    # retrieve just the desired two PCs
    just_PCs = PCs[:,[PCa,PCb]]
    # create the convex hull
    hull = ConvexHull(just_PCs)
    # retreieve hull points
    hull_points = just_PCs[hull.vertices]
    # find two random points in 2D convex hull
    # these are the terminal points of the line
    term_pts = dist_in_hull(hull_points,2)
    # these are the values of the coordinate points
    start_x = term_pts[0][0]
    start_y = term_pts[0][1]
    end_x = term_pts[1][0]
    end_y = term_pts[1][0]

    return start_x, start_y, end_x, end_y

def save_model_values(param_var_dict, model_results_path, n_models):
    """
    Given parameter values, generate and save values to create models

    Inputs
    param_var_dict: a dictionary of parameter variables and values
    model_results_path: path to create a folder to save model values
    n_models: the desired number of model values to save

    Outputs
    No return, but save desirved number of model values in specified folder as csv files
    """

    if not os.path.exists(model_results_path): # check if the folder exists
        # create the folder if it doesn't exist
        os.makedirs(model_results_path)

    # unpack parameter variable list
    parameter_name=param_var_dict["parameter_name"]
    morphospace_file=param_var_dict["morphospace_file"]
    PC_val_file=param_var_dict["PC_val_file"]
    PCa=param_var_dict["PCa"]
    PCb=param_var_dict["PCb"]
    grid_density=param_var_dict["grid_density"]
    low_lf_num=param_var_dict["low_lf_num"]
    high_lf_num=param_var_dict["high_lf_num"]
    low_juv_len=param_var_dict["low_juv_len"]
    high_juv_len=param_var_dict["high_juv_len"]
    low_adult_len=param_var_dict["low_adult_len"]
    high_adult_len=param_var_dict["high_adult_len"]
    lf_curve=param_var_dict["lf_curve"]
    lf_curl=param_var_dict["lf_curl"]
    max_down_curl=param_var_dict["max_down_curl"]
    max_up_curl=param_var_dict["max_up_curl"]
    ang_min_limit=param_var_dict["ang_min_limit"]
    ang_max_limit=param_var_dict["ang_max_limit"]
    phyllo_option=param_var_dict["phyllo_option"]

    for m in range(n_models):
    
        # select number of leaves
        n = select_leaf_number(low_lf_num, high_lf_num)
    
        # select phyllotactic angle
        phyllo_ang = select_phyllotaxy_angle(phyllo_option)
    
        # select leaf lengths
        juv_len, adult_len, node_max = select_leaf_lengths(low_juv_len, high_juv_len, low_adult_len, high_adult_len)
    
        # select leaf surface variables
        leaf_curve, leaf_curl, leaf_tip, leaf_mid = select_leaf_surface_variables(lf_curve, lf_curl, max_down_curl, max_up_curl)
    
        # select leaf elevation angles
        angle_min, angle_max = select_leaf_elevation_angles(ang_min_limit, ang_max_limit)
    
        # select morphospace points
        start_x, start_y, end_x, end_y = select_morphospace_points(morphospace_file, PC_val_file, PCa, PCb)
    
        model_var_list = [
            "n",
            "phyllo_ang",
            "juv_len",
            "adult_len",
            "node_max",
            "leaf_curve",
            "leaf_curl",
            "leaf_tip",
            "leaf_mid",
            "angle_min",
            "angle_max",
            "start_x",
            "start_y",
            "end_x",
            "end_y"
           ]
    
        model_val_list = [
            n,
            phyllo_ang,
            juv_len,
            adult_len,
            node_max,
            leaf_curve,
            leaf_curl,
            leaf_tip,
            leaf_mid,
            angle_min,
            angle_max,
            start_x,
            start_y,
            end_x,
            end_y
            ]

        # convert values into lists
        model_val_list = [[x] for x in model_val_list]
    
        # convert parameter names and values into dictionary
        model_var_dict = dict(zip(model_var_list, model_val_list))

        # convert dict into df
        df = pd.DataFrame.from_dict(model_var_dict)
        
        # save param values
        df.to_csv("./" + model_results_path + "/m" + str(m) + "_" + parameter_name + ".csv", index=False)

def set_model_values(path):
    """
    Given a model values file, read in and set model values

    Inputs
    path: a path to a model value .csv file

    Outputs
    model_var_dict: a dictionary of model values
    The function sets global values for model variables
    """

    # read in model values from csv
    df = pd.read_csv(path)
    
    # create variables for parameters
    for column in df.columns:
        globals()[column] = df[column].item()
    
    # create a dictionary of model values to return
    model_var_list = [
        "n",
        "phyllo_ang",
        "juv_len",
        "adult_len",
        "node_max",
        "leaf_curve",
        "leaf_curl",
        "leaf_tip",
        "leaf_mid",
        "angle_min",
        "angle_max",
        "start_x",
        "start_y",
        "end_x",
        "end_y"
       ]
    
    model_val_list = [
        n,
        phyllo_ang,
        juv_len,
        adult_len,
        node_max,
        leaf_curve,
        leaf_curl,
        leaf_tip,
        leaf_mid,
        angle_min,
        angle_max,
        start_x,
        start_y,
        end_x,
        end_y
        ]
    
    # convert model names and values into dictionary
    model_var_dict = dict(zip(model_var_list, model_val_list))

    return model_var_dict

# 1. Set parameters

### Save parameter names and values

In [65]:
# name of parameter set
parameter_name = "nahuatlleaftest"

# morphospace and PC values to synthesize leaf shape
morphospace_file = "nahuatl_morphospace_may_10_2025.pkl" # PCA model file, .pkl file
PC_val_file = "nahuatl_PCs_may_10_2025.npy" # empirical PCA values from the PCA, .npy file

# select two PC axes to sythesize leaves from
PCa = 0 # index position of first PC (index 0 = PC1)
PCb = 4 # index position of second PC (index 0 = PC1)

# set density of grid to sample points within leaves
grid_density = 0.01 # density to sample points on leaves

# select the range of leaf numbers to uniformly sample from
low_lf_num = 3 # inclusive
high_lf_num = 10 # up to, not including

# set relative height of first leaf in series
low_juv_len = 0.2 # inclusive
high_juv_len = 1 # not including

# set height of last leaf relative to one
low_adult_len = 0.2 # inclusive
high_adult_len = 1 # not including

# set maximum scaling values for leaf curvature and leaf curling
lf_curve = 0.8 # max scaling of leaf curvature (the interior of the leaf)
lf_curl = 0.8 # max scaling of leaf curl (across length of leaf)

# set maximum displacement at leaf tip (y value), relative to -1, 1
max_down_curl = -1 
max_up_curl = 1

# set minimum and maximum limits of leaf angles
ang_min_limit = 10 # max possible min angle
ang_max_limit = 90 # max possible angle

# set phyllotaxy angle option (in degrees)
# select from options below:
# 0 = [-137.5077640500378546463487,137.5077640500378546463487]
# 1 = [-137.5077640500378546463487,137.5077640500378546463487,90,180]
# 2 = [-137.5077640500378546463487,137.5077640500378546463487,90,180,<random -360 to +360>]
# 3 = [<random -360 to +360>]
phyllo_option = 2

# list of parameter names
param_name_list = [
    "parameter_name", 
    "morphospace_file", 
    "PC_val_file",
    "PCa", 
    "PCb",
    "grid_density",
    "low_lf_num",
    "high_lf_num",
    "low_juv_len",
    "high_juv_len",
    "low_adult_len",
    "high_adult_len",
    "lf_curve",
    "lf_curl",
    "max_down_curl",
    "max_up_curl",
    "ang_min_limit",
    "ang_max_limit",
    "phyllo_option"
]

# list of parameter values
param_val_list = [
    parameter_name, 
    morphospace_file, 
    PC_val_file,
    PCa, 
    PCb,
    grid_density,
    low_lf_num,
    high_lf_num,
    low_juv_len,
    high_juv_len,
    low_adult_len,
    high_adult_len,
    lf_curve,
    lf_curl,
    max_down_curl,
    max_up_curl,
    ang_min_limit,
    ang_max_limit,
    phyllo_option
]


### Save parameters to file

In [66]:
# save the parameters to file
save_params(keys=param_name_list, values=param_val_list, path = "./" + parameter_name + "_params.csv")
    

# 2. Calculate model values

### Read in parameter values

In [67]:
# name of parameter set
parameter_name = "nahuatlleaftest"

param_path = "./" + parameter_name + "_params.csv"
param_var_dict = set_params(param_path)

### Generate model values

In [68]:
# specify the number of desired model values
n_models = 1000

# specify the folder to save model values
model_values_path = "model_values"

# save model values as csv files to folder
save_model_values(param_var_dict, model_values_path, n_models)


# 3. Generate models

In [70]:
# specify path 
model_values_path = "model_values"

# get file names of model values
file_names = os.listdir(model_values_path) 

# make sure no hidden files
for i in range(len(file_names)): 
    if file_names[i][0]==".":
        file_names.remove(i)

# select a random model value file
file = np.random.choice(file_names)

# retrieve parameter file name from model values string
# read in and set parameter values
param_var_dict = set_params(file[file.index("_")+1:-4]+"_params.csv")

# read in and set model values
model_var_dict = set_model_values(model_values_path+"/"+file)

print(param_var_dict)
print(model_var_dict)

{'parameter_name': 'nahuatlleaftest', 'morphospace_file': 'nahuatl_morphospace_may_10_2025.pkl', 'PC_val_file': 'nahuatl_PCs_may_10_2025.npy', 'PCa': 0, 'PCb': 4, 'grid_density': 0.01, 'low_lf_num': 3, 'high_lf_num': 10, 'low_juv_len': 0.2, 'high_juv_len': 1, 'low_adult_len': 0.2, 'high_adult_len': 1, 'lf_curve': 0.8, 'lf_curl': 0.8, 'max_down_curl': -1, 'max_up_curl': 1, 'ang_min_limit': 10, 'ang_max_limit': 90, 'phyllo_option': 2}
{'n': 6, 'phyllo_ang': 137.50776405003785, 'juv_len': 0.605800652531975, 'adult_len': 0.8357566272724501, 'node_max': 0.5070398751753555, 'leaf_curve': 0.3170151445105089, 'leaf_curl': 0.0174714874236847, 'leaf_tip': 0.4098422613078127, 'leaf_mid': 0.0148346323447654, 'angle_min': 5.374107219658468, 'angle_max': 52.133331386983784, 'start_x': 0.0023372153254273, 'start_y': -0.0547876652647715, 'end_x': 0.1727649619404735, 'end_y': 0.1727649619404735}
